In [1]:
import pyspark as spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1725467168748_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, lit, unix_timestamp, to_timestamp, format_string, expr
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, DoubleType, LongType
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql.functions import col, concat, lit, unix_timestamp, to_timestamp, format_string, expr

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc=spark.SparkContext

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number
from pyspark.sql.functions import dense_rank

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ETL_PROJECT_Spark').master("local").getOrCreate()
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# READ THE DATA TO SPARK

In [7]:
src_atm_trans=spark.read.csv('/user/root/SRC_ATM_TRANS/src_atm_trans.csv',header=False,inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# looking the first 10 rows
src_atm_trans.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+------+---+--------+---+---------------+-----------+-------------------+----+----+------+------+----+------------------+----+----------+----+----+------+------+-------+--------------+------+----+----+----+----+-----+----+----+-------+--------------------+
| _c0|    _c1|_c2|   _c3|_c4|     _c5|_c6|            _c7|        _c8|                _c9|_c10|_c11|  _c12|  _c13|_c14|              _c15|_c16|      _c17|_c18|_c19|  _c20|  _c21|   _c22|          _c23|  _c24|_c25|_c26|_c27|_c28| _c29|_c30|_c31|   _c32|                _c33|
+----+-------+---+------+---+--------+---+---------------+-----------+-------------------+----+----+------+------+----+------------------+----+----------+----+----+------+------+-------+--------------+------+----+----+----+----+-----+----+----+-------+--------------------+
|2017|January|  1|Sunday|  0|  Active|  1|            NCR| NÃƒÂ¦stved|        Farimagsvej|   8|4700|55.233|11.763| DKK|        MasterCard|5643|Withdrawal|null|null| 55.23|11.761|

In [9]:
# changing the schema
new_schema=StructType([StructField('year',IntegerType(),True),
                      StructField('month',StringType(),True),
                      StructField('day',IntegerType(),True),
                      StructField('weekday',StringType(),True),
                      StructField('hour',IntegerType(),True),
                      StructField('atm_status',StringType(),True),
                      StructField('atm_id',StringType(),True),
                      StructField('atm_manufacturer',StringType(),True),
                      StructField('atm_location',StringType(),True),
                      StructField('atm_streetname',StringType(),True),
                      StructField('atm_street_number',IntegerType(),True),
                      StructField('atm_zipcode',IntegerType(),True),
                      StructField('atm_lat',DoubleType(),True),
                      StructField('atm_lon',DoubleType(),True),
                      StructField('currency',StringType(),True),
                      StructField('card_type',StringType(),True),
                      StructField('transaction_amount',IntegerType(),True),
                      StructField('service',StringType(),True),
                      StructField('message_code',StringType(),True),
                      StructField('message_text',StringType(),True),
                      StructField('weather_lat',DoubleType(),True),
                      StructField('weather_lon',DoubleType(),True),
                      StructField('weather_city_id',IntegerType(),True),
                      StructField('weather_city_name',StringType(),True),
                      StructField('temp',DoubleType(),True),
                      StructField('pressure',IntegerType(),True),
                      StructField('humidity',IntegerType(),True),
                      StructField('wind_speed',IntegerType(),True),
                      StructField('wind_deg',IntegerType(),True),
                      StructField('rain_3h',DoubleType(),True),
                      StructField('clouds_all',IntegerType(),True),
                      StructField('weather_id',IntegerType(),True),
                      StructField('weather_main',StringType(),True),
                      StructField('weather_description',StringType(),True),])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# infering the manual schema and reading the data
df=spark.read.csv('/user/root/SRC_ATM_TRANS/src_atm_trans.csv',header=False,schema=new_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+------------------+------------------+----------+------------+------------+-----------+-----------+---------------+-----------------+------+--------+--------+----------+--------+-------+----------+----------+------------+--------------------+
|year|  month|day|weekday|hour|atm_status|atm_id|atm_manufacturer|atm_location|     atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|currency|         card_type|transaction_amount|   service|message_code|message_text|weather_lat|weather_lon|weather_city_id|weather_city_name|  temp|pressure|humidity|wind_speed|wind_deg|rain_3h|clouds_all|weather_id|weather_main| weather_description|
+----+-------+---+-------+----+----------+------+----------------+------------+-------------------+-----------------+-----------+-------+-------+--------+------------------+------------------+----------+---

In [12]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

# CREATING FACT & DIMENSION TABLE

In [13]:
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Dimension_Table: Location Dimension

In [15]:
# Let's create a DataFrame location, select specific columns, and ensure distinct records:
location=df.select('atm_location','atm_streetname','atm_street_number','atm_zipcode','atm_lat','atm_lon').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Creating LOCATION_ID as primary key in the dataframe 
location_dim=location.select(row_number().over(Window.orderBy(location['atm_location'])).alias('location_id'),"*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [145]:
# Display first 5 rows to check if command run above are properly executed as per plan
location_dim.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
|location_id|        atm_location|atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|
+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
|          1|             Aabybro|  ÃƒËœstergade|                6|       9440| 57.162|   9.73|
|          2|      Aalborg Hallen|  Europa Plads|                4|       9000| 57.044|  9.913|
|          3|Aalborg Storcente...|      Hobrovej|              452|       9200| 57.005|  9.876|
|          4|Aalborg Storcente...|      Hobrovej|              452|       9200| 57.005|  9.876|
|          5|         Aalborg Syd|      Hobrovej|              440|       9200| 57.005|  9.881|
+-----------+--------------------+--------------+-----------------+-----------+-------+-------+
only showing top 5 rows

In [24]:

# Now renaming columns name as per DIM_LOCATION given in ETL Project: Schema for Dimensions and Fact
DIM_LOCATION=location_dim.withColumnRenamed('atm_location','location')\
                         .withColumnRenamed('atm_streetname','street_name')\
                         .withColumnRenamed('atm_street_number','street_number')\
                         .withColumnRenamed('atm_zipcode','zipcode')\
                         .withColumnRenamed('atm_lat','latitude')\
                         .withColumnRenamed('atm_lon','longitude')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
DIM_LOCATION.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+------------------+-------------+-------+--------+---------+
|location_id|            location|       street_name|street_number|zipcode|latitude|longitude|
+-----------+--------------------+------------------+-------------+-------+--------+---------+
|          1|             Aabybro|      ÃƒËœstergade|            6|   9440|  57.162|     9.73|
|          2|      Aalborg Hallen|      Europa Plads|            4|   9000|  57.044|    9.913|
|          3|Aalborg Storcente...|          Hobrovej|          452|   9200|  57.005|    9.876|
|          4|Aalborg Storcente...|          Hobrovej|          452|   9200|  57.005|    9.876|
|          5|         Aalborg Syd|          Hobrovej|          440|   9200|  57.005|    9.881|
|          6|           AalbÃƒÂ¦k|        Centralvej|            5|   9982|  57.593|   10.412|
|          7|              Aarhus|    SÃƒÂ¸nder Alle|           11|   8000|  56.153|   10.206|
|          8|              Aarhus|        Ceres By

In [25]:
DIM_LOCATION.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [143]:
DIM_LOCATION.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- location_id: integer (nullable = false)
 |-- location: string (nullable = true)
 |-- street_name: string (nullable = true)
 |-- street_number: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)

## Dimension_Table: ATM Dimension

In [132]:
# Selecting required columns from the main dataframe into atm
atm=df.select("atm_id", "atm_manufacturer", 'atm_lat', 'atm_lon').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
atm.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------+-------+-------+
|atm_id|atm_manufacturer|atm_lat|atm_lon|
+------+----------------+-------+-------+
|    54|             NCR| 56.745|  8.949|
|    34|             NCR| 57.034|  9.908|
|    31|             NCR| 55.398| 11.342|
|    85| Diebold Nixdorf| 55.676| 12.571|
|    82|             NCR| 57.494|  9.838|
+------+----------------+-------+-------+
only showing top 5 rows

In [134]:
atm.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [135]:
# Join location_dim through left join on atm 
atm = atm.join(location_dim, on = ['atm_lat', 'atm_lon'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
# After joining data look like this 
atm.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+------+----------------+-----------+--------------+------------------+-----------------+-----------+
|atm_lat|atm_lon|atm_id|atm_manufacturer|location_id|  atm_location|    atm_streetname|atm_street_number|atm_zipcode|
+-------+-------+------+----------------+-----------+--------------+------------------+-----------------+-----------+
| 56.745|  8.949|    54|             NCR|         22|         Durup|            Torvet|                4|       7870|
| 57.494|  9.838|    82|             NCR|         83|Skallerup Klit|    Nordre Klitvej|               21|       9800|
| 55.398| 11.342|    31|             NCR|         89|      Slagelse|Mariendals AllÃƒÂ¨|               29|       4200|
| 55.398| 11.342|    31|             NCR|         88|      Slagelse|   Mariendals Alle|               29|       4200|
| 55.676| 12.571|    85| Diebold Nixdorf|         63|  KÃƒÂ¸benhavn|    Regnbuepladsen|                5|       1550|
+-------+-------+------+----------------+-----------+---

In [137]:
atm.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

156

In [138]:
# Changing atm_id to atm_number as we have to create one new column as atm_id as primary key
atm_dim=atm.withColumnRenamed('atm_id','atm_number').withColumnRenamed('location_id','atm_location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [139]:
# selecting the required columns and making sure records are distinct
atm_dim = atm_dim.select('atm_number', 'atm_manufacturer', 'atm_location_id').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
atm_dim.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+---------------+
|atm_number|atm_manufacturer|atm_location_id|
+----------+----------------+---------------+
|         8|             NCR|             30|
|        97|             NCR|             51|
|       113| Diebold Nixdorf|             89|
|        51|             NCR|             29|
|        64|             NCR|             43|
+----------+----------------+---------------+
only showing top 5 rows

In [141]:
DIM_ATM= atm_dim.select(row_number().over(Window.orderBy(atm_dim["atm_number"])).alias("atm_id"),"*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
DIM_ATM.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+----------------+---------------+
|atm_id|atm_number|atm_manufacturer|atm_location_id|
+------+----------+----------------+---------------+
|     1|         1|             NCR|             74|
|     2|        10|             NCR|             76|
|     3|       100|             NCR|             87|
|     4|       100|             NCR|             56|
|     5|       100|             NCR|             86|
+------+----------+----------------+---------------+
only showing top 5 rows

In [110]:
DIM_ATM.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

156

## Dimension_Table: Date Dimension

In [52]:
# creating a df and selecting required columns
date = df.select('year', 'month', 'day', 'hour', 'weekday')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [53]:
date.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+-------+
|year|  month|day|hour|weekday|
+----+-------+---+----+-------+
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
|2017|January|  1|   0| Sunday|
+----+-------+---+----+-------+
only showing top 20 rows

In [54]:
date = date.withColumn('full_date_time', to_timestamp(concat(date.year, lit('-'), date.month, lit('-'),date.day, lit(' '), date.hour), 'yyyy-MMMM-d H'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
date.show(5)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---+----+-------+-------------------+
|year|  month|day|hour|weekday|     full_date_time|
+----+-------+---+----+-------+-------------------+
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
|2017|January|  1|   0| Sunday|2017-01-01 00:00:00|
+----+-------+---+----+-------+-------------------+
only showing top 5 rows

In [56]:
date.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [57]:
# selecting the columns and making sure that records are distinct
date = date.select('full_date_time', 'year', 'month', 'day', 'hour', 'weekday').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
# Creating DIM_DATE as per schema requirement 
DIM_DATE = date.select(row_number().over(Window.orderBy(date["full_date_time"])).alias('date_id'), '*')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
DIM_DATE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+----+-------+---+----+-------+
|date_id|     full_date_time|year|  month|day|hour|weekday|
+-------+-------------------+----+-------+---+----+-------+
|      1|2017-01-01 00:00:00|2017|January|  1|   0| Sunday|
|      2|2017-01-01 01:00:00|2017|January|  1|   1| Sunday|
|      3|2017-01-01 02:00:00|2017|January|  1|   2| Sunday|
|      4|2017-01-01 03:00:00|2017|January|  1|   3| Sunday|
|      5|2017-01-01 04:00:00|2017|January|  1|   4| Sunday|
+-------+-------------------+----+-------+---+----+-------+
only showing top 5 rows

In [60]:
DIM_DATE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

## Dimension_Table: Card Type Dimension

In [129]:
card_type = df.select('card_type').distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
card_type.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|           card_type|
+--------------------+
|Visa Dankort - on-us|
|  Mastercard - on-us|
|         HÃƒÂ¦vekort|
|            VisaPlus|
|     Dankort - on-us|
+--------------------+
only showing top 5 rows

In [62]:
DIM_CARD_TYPE =card_type.select(row_number().over(Window.orderBy(card_type["card_type"])).alias("card_type_id"),"*")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
DIM_CARD_TYPE.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-------------------+
|card_type_id|          card_type|
+------------+-------------------+
|           1|             CIRRUS|
|           2|            Dankort|
|           3|    Dankort - on-us|
|           4|        HÃƒÂ¦vekort|
|           5|HÃƒÂ¦vekort - on-us|
+------------+-------------------+
only showing top 5 rows

In [63]:
DIM_CARD_TYPE.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

## Fact_Table : FACT_ATM_TRANS

In [111]:
fact_loc = df.withColumnRenamed('atm_location','location')\
    .withColumnRenamed('atm_streetname','street_name')\
    .withColumnRenamed('atm_street_number','street_number')\
    .withColumnRenamed('atm_zipcode','zipcode')\
    .withColumnRenamed('atm_lat','latitude')\
    .withColumnRenamed('atm_lon','longitude')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
# joining original dataframe with DIM_LOCATION
fact_loc = fact_loc.join(DIM_LOCATION, on = ['location', 'street_name', 'street_number', 'zipcode', 'latitude', 'longitude'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
fact_loc.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [115]:
fact_loc = fact_loc.withColumnRenamed('atm_id', 'atm_number').withColumnRenamed('location_id','atm_location_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
fact_loc.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['location', 'street_name', 'street_number', 'zipcode', 'latitude', 'longitude', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'atm_number', 'atm_manufacturer', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_location_id']

In [117]:
# joining the dataframe with DIM_ATM
fact_atm = fact_loc.join(DIM_ATM, on = ['atm_number', 'atm_manufacturer', 'atm_location_id'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [118]:
DIM_ATM.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_id', 'atm_number', 'atm_manufacturer', 'atm_location_id']

In [119]:
fact_atm.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['atm_number', 'atm_manufacturer', 'atm_location_id', 'location', 'street_name', 'street_number', 'zipcode', 'latitude', 'longitude', 'year', 'month', 'day', 'weekday', 'hour', 'atm_status', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id']

In [120]:
# performing necessary transformations, same as done to atm table
fact_atm = fact_atm.withColumnRenamed('atm_location_id', 'weather_loc_id')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [121]:
# joining the dataframe with DIM_DATE
fact_date = fact_atm.join(DIM_DATE, on = ['year', 'month', 'day', 'hour', 'weekday'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
fact_date.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'street_name', 'street_number', 'zipcode', 'latitude', 'longitude', 'atm_status', 'currency', 'card_type', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time']

In [123]:
# joining the dataframe with DIM_CARD_TYPE
fact_atm_trans = fact_date.join(DIM_CARD_TYPE, on = ['card_type'], how = "left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [124]:
fact_atm_trans.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'street_name', 'street_number', 'zipcode', 'latitude', 'longitude', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time', 'card_type_id']

In [125]:
# creating primary key of fact table
FACT_ATM_TRANS = fact_atm_trans.withColumn("trans_id", row_number().over(Window.orderBy('date_id')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
# viewing the list of columns
FACT_ATM_TRANS.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['card_type', 'year', 'month', 'day', 'hour', 'weekday', 'atm_number', 'atm_manufacturer', 'weather_loc_id', 'location', 'street_name', 'street_number', 'zipcode', 'latitude', 'longitude', 'atm_status', 'currency', 'transaction_amount', 'service', 'message_code', 'message_text', 'weather_lat', 'weather_lon', 'weather_city_id', 'weather_city_name', 'temp', 'pressure', 'humidity', 'wind_speed', 'wind_deg', 'rain_3h', 'clouds_all', 'weather_id', 'weather_main', 'weather_description', 'atm_id', 'date_id', 'full_date_time', 'card_type_id', 'trans_id']

In [127]:
# selecting and arranging only the required columns according to the target model
FACT_ATM_TRANS = FACT_ATM_TRANS.select('trans_id', 'atm_id', 'weather_loc_id', 'date_id', 'card_type_id', 
'atm_status', 'currency', 'service', 'transaction_amount', 'message_code', 'message_text', 'rain_3h', 
'clouds_all', 'weather_id', 'weather_main', 'weather_description')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
FACT_ATM_TRANS.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

## Loading Data into aws s3 bucket

In [152]:
# Writing DIM_LOCATION to S3
DIM_LOCATION.write.format('csv').option('header', 'false').save('s3://etl-atm-buk/dim_location/DIM_LOCATION.csv', mode='overwrite')

# Writing DIM_ATM to s3
DIM_ATM.write.format('csv').option('header', 'false').save('s3://etl-atm-buk/dim_atm/DIM_ATM.csv', mode='overwrite')

# Writing DIM_DATE to S3
DIM_DATE.write.format('csv').option('header', 'false').save('s3://etl-atm-buk/dim_date/DIM_DATE.csv', mode='overwrite')

# Writing DIM_CARD_TYPE to S3
DIM_CARD_TYPE.write.format('csv').option('header', 'false').save('s3://etl-atm-buk/dim_card_type/DIM_CARD_TYPE.csv', mode='overwrite')

# Writing FACT_ATM_TRANS to S3
FACT_ATM_TRANS.write.format('csv').option('header', 'false').save('s3://etl-atm-buk/fact_atm_trans/FACT_ATM_TRANS', mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…